In [1]:
#from jinja2 import FileSystemLoader, Environment
from glom import glom
import numpy as np

import sys,os,re

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/Model/v3")
sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/lib")

from oos_test import OosTest
from strategy import Strategy
from candidate import Candidate
from prototype import Prototype
from proto_config import ProtoConfig
from prototype_parameter import PrototypeParameter
from entry_filter import EntryFilter
from poi import Poi
from session import Session as TradeSession
from base import Base, Session

from ats_tools import hhmm2mins, mins2hhmm, add2hhmm
from db_query import connectDB,nextPrototypeId,queryPrototype,queryPrototypeParams
from db_query import queryEntryFilter,queryPoi,querySession

from prototype_tools import getPrototypeSetup
from prototype_tools import generatePrototypeCode,generateJclCode

#dp = {}


In [ ]:
'''
def connectDB():
    db = Session()
    db_url = os.environ["DB_HEROKU"]

    engine = create_engine(db_url)
    Base.metadata.bind = engine

    DBSession = sessionmaker(bind=engine)
    session = DBSession()
    return session
'''

In [ ]:
'''
def nextPrototypeId(dbh):
    #return dbh.query(Prototype).filter(Prototype.id > 0).order_by(Prototype.id).all()
    return dbh.query(ProtoConfig).filter(ProtoConfig.id > 0).order_by(ProtoConfig.id).all()
dbh = connectDB()
print(nextPrototypeId(dbh)[0].id)
'''

In [ ]:
'''
def queryPrototype(dbh,id):
    return dbh.query(ProtoConfig).filter(ProtoConfig.id == id).all()
    #return dbh.query(Prototype).filter(Prototype.id == id).all()
#dbh = connectDB()
#print(queryPrototype(dbh,1)[0].symbol)
'''

In [ ]:
'''
def queryPrototypeParams(dbh,id):
    return dbh.query(PrototypeParameter).filter(PrototypeParameter.proto_id == id).all()
#dbh = connectDB()
#print(queryPrototypeParams(dbh,1)[0].name)
'''

In [ ]:
'''
def queryEntryFilter(dbh,id):
    return dbh.query(EntryFilter).filter(EntryFilter.id == id).one_or_none()
#dbh = connectDB()
#print(queryEntryFilter(dbh,1).long_logic)
'''

In [ ]:
'''
def queryPoi(dbh,id):
    return dbh.query(Poi).filter(Poi.id == id).one_or_none()
#dbh = connectDB()
#print(queryPoi(dbh,1).long_logic)
'''

In [ ]:
'''
def querySession(dbh,id):
    return dbh.query(TradeSession).filter(TradeSession.id==id).one_or_none()
#dbh = connectDB()
#print(f'{querySession(dbh,8).session_start} :: {querySession(dbh,8).session_end}')
'''

In [ ]:
'''
def render_from_template(directory, template_name, **kwargs):
    loader = FileSystemLoader(directory)
    env = Environment(loader=loader)
    template = env.get_template(template_name)
    return template.render(**kwargs)
'''

In [ ]:
def formatCommentStrings(setup):
    tf_arr = []
    for tf in setup['timeframes']:
        tf_arr.append(f"Timeframe {tf['ds']} = {tf['tf']} {tf['unit']}")
    tf_str = ('\n').join(tf_arr)
    desc={}
    desc['chart_setup'] = f"""Chart Setup:
      symbol = {setup['symbol']}
      {tf_str}
      Fitness Function = {setup['fitnessFunc']}
      maxDaysBack = {setup['maxBarsBack']}
      session name = {setup['session_name']}
      start date = {setup['start_dt']}
      end date = {setup['end_dt']}"""
    desc['prototype_info'] = f'''Prototype Info::
      dataSet = {setup['data_set']}
      dataBlock = {setup['data_block']}    
    '''
    
    return desc

In [ ]:
'''
def addInputRange(p):
    inp_def = {}
    inp_def['type'] = 'range'
    inp_def['dtype'] = p.data_type
    inp_def['params'] = p.value
    if p.data_type == 'double':
        st,end,step = [float(x) for x in p.value.split(',')]
        inp_def['value'] = [format(x,'.2f') for x in np.arange(st,end+step,step)]
    else:
        st,end,step = [int(x) for x in p.value.split(',')]
        inp_def['value'] = [int(x) for x in np.arange(st,end+step,step)]
    return inp_def
'''

In [ ]:
'''
def addInputList(p):
    inp_def = {}
    inp_def['type'] = 'list'
    inp_def['dtype'] = p.data_type
    inp_def['value'] = [int(x) for x in p.value.split(',')]
    return inp_def
'''

In [ ]:
'''
def addInputVar(p):
    inp_def = {}
    inp_def['type'] = 'var'
    inp_def['dtype'] = p.data_type
    inp_def['value'] = [p.value,]
    return inp_def
'''

In [ ]:
'''
def warn(str):
    print(f'WARNING: {str}')
'''

In [ ]:
'''
def parseInput(param):
    if param.input_type == 'range':
        return param.name, addInputRange(param)
    elif param.input_type == 'list':
        return param.name, addInputList(param)
    elif param.input_type == 'var':
        return param.name, addInputVar(param)
    else:
        warn(f'input_type({p.input_type}) not coded for, id={p.id}')
'''

In [ ]:
'''
def setSessionVars(id):
    sess = querySession(dbh,id)
    return sess.session_start, sess.session_end
'''

In [ ]:
'''
def setDataSeries(series):
    # 1:@RTY.D:120:m,2:@RTY.D:1440:m
    ds = []
    for s in series.split(','):
        sn,symbol,tf,tf_unit = s.split(':')
        tf_dict = {'sn': sn, 'symbol': symbol, 'tf': tf, 'tf_unit': tf_unit}
        ds.append(tf_dict)
    return ds

#series = '1:@RTY.D:120:m,2:@RTY.D:1440:m'
#print(setDataSeries(series))
'''

In [ ]:
'''
import datetime

def formatDate(from_dt):
    return datetime.datetime.strftime(from_dt,'%m/%d/%Y')
'''

In [ ]:
'''
def getConfiguration(dbh,id):
    #proto = queryPrototype(dbh,id)[0]
    proto = queryPrototype(dbh,id)
    params = queryPrototypeParams(dbh,id)
    setup = {}
    setup['proto_id'] = proto.id
    setup['template_version'] = proto.template_version
    setup['symbol'] = proto.symbol
    setup['data_set'] = proto.data_set
    setup['data_block'] = proto.data_block
    setup['chart_series'] = proto.chart_series
    setup['fitnessFunc'] = proto.fitness_function
    setup['maxBarsBack'] = proto.max_days_back
    setup['oosPercentLast'] = 20
    setup['session_num'] = proto.session_id
    setup['session_name'] = f'session {proto.session_id}'
    setup['start_dt'] = formatDate(proto.start_dt)
    setup['end_dt'] = formatDate(proto.end_dt)
    setup['chart_series'] = proto.chart_series

    setup['sess_start'],setup['sess_end'] = setSessionVars(setup['session_num'])
    setup['data_series'] = setDataSeries(setup['chart_series'])
    
    setup['opt_inputs'] = {}
    setup['param_vars'] = {}
    for p in params:
        param_name,param_setup = parseInput(p)
        setup['opt_inputs'][param_name] = param_setup
    setup['timeframes'] = []
    for ds in setup['chart_series'].split(','):
        i,symbol,tf,unit = ds.split(':')
        tf_d = {'ds': i, 'symbol': symbol, 'tf': tf, 'unit': unit}
        setup['timeframes'].append(tf_d)
        
    dir = f"{ats_dir}/Data/OptimizationApiCode/"
    setup['jcl_file'] = f"{dir}/proto_{setup['proto_id']}.jcl"
    dir = f'{ats_dir}/Data/PrototypeCode/'
    setup['strategy_name'] = f"proto_{setup['proto_id']}"
    setup['strategy_file'] = f"{dir}/{setup['strategy_name']}"

    return setup
'''

In [ ]:
'''
def setHeader(setup):
    return f'''      prototype: { setup['proto_id'] }
      template: { setup['template_version'] }
      job file: { setup['jcl_file'] }
      poi IDs     : { glom(setup,'opt_inputs.poi.value',default=None) }
      filter 1 IDs: { glom(setup,'opt_inputs.filter_1.value',default=None) }
      filter 2 IDs: { glom(setup,'opt_inputs.filter_2.value',default=None) }
'''

In [ ]:
'''
def filterLogic(setup):
    filters = {}
    ds = 1
    while True:
        ids = glom(setup,f'opt_inputs.filter_{ds}.value',default=None)
        if ids is None:
            break
        if isinstance(ids,str):
            ids = ids.split(',')
        idx = 1
        filters[ds] = {}
        for id in ids:
            q = queryEntryFilter(dbh,id)
            try:
                #print(f"long_code{id}: {q.long_logic}")
                long_code = q.long_logic.format(p1=f'filter_{ds}_n1',p2=f'filter_{ds}_n2',fn=ds)
            except AttributeError:
                long_code = None
            try:
                #print(f"short_code{id}: {q.short_logic}")
                short_code = q.short_logic.format(p1=f'filter_{ds}_n1',p2=f'filter_{ds}_n2',long=f'{long_code}',fn=ds)
            except AttributeError:
                short_code = None

            #d_filter = {'fid': id, 
            #            'long': long_code,
            #            'short': short_code}
            filters[ds][idx] = {'fid': id, 
                        'long': long_code,
                        'short': short_code}
            idx+=1
            
        ds+=1
    return filters
'''

In [ ]:
'''
def lsbLogic(setup):
    if glom(setup,'opt_inputs.lsb.value',default=None) != 4: return
    setup['opt_inputs']['lsb']['value'] = 3
    logic = f"""
        switch(dow)
        begin
            case 1:
                lsb = mon_lsb;
            case 2:
                lsb = tue_lsb;
            case 3:
                lsb = wed_lsb;
            case 4:
                lsb = thu_lsb;
            case 5:
                lsb = fri_lsb;
        end;
    """
    return logic
'''

In [ ]:
'''
def poiLogic(setup):
    id_list = glom(setup,'opt_inputs.poi.value',default=None)
    logic = []
    if id_list == '' or id_list == None: return logic
    ids = id_list
    if isinstance(id_list, str):
        ids = id_list.split(',')     
    if len(ids) == 0: return []
    for id in ids:
        q = queryPoi(dbh,id)
        if not q:
            logic = None
            warn(f"poiLogic: No POI logic for id: {id}")
            continue
        #pprint(q)
        logic.append({'poi_id': id, 'common': q.common_logic, 'long': q.long_logic, 'short': q.short_logic})
        if(q.vars):
            for v in q.vars.split(';'):
                name,dtype,value = v.split(':')
                setup['param_vars'][name] = {'d_type': dtype, 'value': value}
                #'opt_inputs': {'filter_1': {'dtype': 'int',
                #             'type': 'distinct',
                #             'value': [2, 5, 6]},
            #pprint(setup)      #'param_vars': 'int:cum_ma_cnt;int:cum_ma',
    return logic
'''

In [ ]:
'''
def calcTimeSegmentSize(start,end,n_segs=3):
    return (hhmm2mins(end)-hhmm2mins(start)) / n_segs
#print(calcTimeSegmentSize(930,1600))
'''

In [ ]:
'''
def timeSegmentLogic(setup):
    #print(f"timeSegmentLogic(setup): timeSegmentLogic({setup})")
    seg=glom(setup,'opt_inputs.time_segment.value',default=None)
    print(f"seg={seg}")
    tseg = calcTimeSegmentSize(setup['sess_start'],setup['sess_end'])
    print(f"tseg={tseg}")
    timeframe = int(setup['timeframes'][0]['tf'])
    print(f"timeframe = {timeframe}")
    sess_start = setup['sess_start']
    sess_end = setup['sess_end']
    print(f"session: {sess_start} - {sess_end}")
    if len(seg)>1:
        print(1)
        print(f"1: add2hhmm(sess_start, tseg)={add2hhmm(sess_start, tseg)}")
        print(f"1: add2hhmm(sess_start, (2*tseg) )={add2hhmm(sess_start, (2*tseg) )}")
        print(f"1: add2hhmm(sess_end,-timeframe)={add2hhmm(sess_end,-timeframe)}")
        print("1 done")
        code = f'''
        Switch (time_segment) begin
          case 0: begin
            time_filter = True;
          end;
          case 1 : begin
            time_filter = time > {sess_start} and time <= {add2hhmm(sess_start, tseg)};
          end;
          case 2 : begin
            time_filter = time > {add2hhmm(sess_start, tseg)} and time <= {add2hhmm(sess_start, (2*tseg) )};
          end;
          case 3 : begin
            time_filter = time > {add2hhmm(sess_start, (2*tseg))} and time < {add2hhmm(sess_end,-timeframe)};
          end;
        end;
        '''
    elif (len(seg) == 1):
        print(2)
        print("len(seg) == 1")
        if seg[0] == '0':
            code = f"time_filter = True;"
        elif seg[0] == '1':
            code = f"time_filter = time > {sess_start} and time <= {add2hhmm(sess_start, tseg)};"
        elif seg[0] == '2':
            code = f"time_filter = time > {add2hhmm(sess_start, tseg)} and time <= {add2hhmm(sess_start, (2*tseg) )};"
        elif seg[0] == '3':
            code = f"time_filter = time > {add2hhmm(sess_start, (2*tseg))} and time < {add2hhmm(sess_end,-timeframe)};"
        else:
            warn(f"timeSegmentLogic({seg}) NOT CODED")
    else:
        print(3)
        code = f"time_filter = True;"
        
    return code

timeSegmentLogic({ 'sess_start': 930, 
                  'sess_end': 1615,
                  'timeframes': [{'ds': '0', 'symbol': '@RTY.D', 'tf': '30', 'unit': 'm'}, {'ds': '1', 'symbol': '@RTY.D', 'tf': '1440', 'unit': 'm'}],
                  'opt_inputs': {'time_segment': {'type': 'range', 'dtype': 'int', 'params': '0,3,1', 'value': [0, 1, 2, 3]}, 
                                'filter_2': {'type': 'list', 'dtype': 'int', 'value': [19, 32]}, 
                                'poi': {'type': 'range', 'dtype': 'int', 'params': '1,8,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8]}, 
                                'natr': {'type': 'range', 'dtype': 'int', 'params': '5,60,5', 'value': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]}, 
                                'fract': {'type': 'range', 'dtype': 'double', 'params': '0.6,3.0,0.15', 'value': ['0.60', '0.75', '0.90', '1.05', '1.20', '1.35', '1.50', '1.65', '1.80', '1.95', '2.10', '2.25', '2.40', '2.55', '2.70', '2.85', '3.00']}, 
                                'filter_1': {'type': 'range', 'dtype': 'int', 'params': '1,40,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]}, 
                                'lsb': {'type': 'var', 'dtype': 'int', 'value': ['1']}, 
                                'filter_2_n1': {'type': 'range', 'dtype': 'int', 'params': '1,20,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, 
                                'filter_1_n1': {'type': 'range', 'dtype': 'int', 'params': '1,20,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, 
                                'filter_1_n2': {'type': 'range', 'dtype': 'int', 'params': '1,20,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, 
                                'filter_2_n2': {'type': 'range', 'dtype': 'int', 'params': '1,20,1', 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}
                               }
                 }
                )
'''

In [ ]:
'''
def stopLossLogic(setup):
    param=glom(setup,'opt_inputs.stop_loss.value',default=None)
    if param == None: return None
    code = "SetStopLoss(stop_loss);"
    return code
'''

In [ ]:
'''
def profitTargetLogic(setup):
    param=glom(setup,'opt_inputs.profit_target.value',default=None)
    if param == None: return None
    if setup:
        code = "SetProfitTarget(profit_target);"
    return code
'''

In [ ]:
'''
def processTemplate(hdr,desc,setup,logic):
    templateLoader = FileSystemLoader(searchpath=f"{ats_dir}/Code/Templates/Prototype/")
    templateEnv = Environment(loader=templateLoader)
    TEMPLATE_FILE = setup['template_version']
    print(f"processTemplate: template file = {TEMPLATE_FILE}")
    template = templateEnv.get_template(TEMPLATE_FILE)
    print(f"desc={desc}")
    print(f"hdr={hdr}")
    outputText = template.render(header=hdr,
                                 chart_setup=desc['chart_setup'],
                                 prototype_info=desc['prototype_info'],
                                 dp_data=dp,
                                 comments=desc,
                                 poi = "more testing for poi",
                                 filters = logic['filters'],
                                 setup=setup,
                                 logic=logic
                                )
    return(outputText)
'''

In [ ]:
'''
def processJclTemplate(setup):
    in_sample_file = f"proto_{setup['proto_id']}_is.csv"
    out_of_sample_file = f"proto_{setup['proto_id']}_oos.csv"
    templateLoader = FileSystemLoader(searchpath=f"{ats_dir}/Code/Templates/JCL/")
    templateEnv = Environment(loader=templateLoader)
    #TEMPLATE_FILE = f"proto_{setup['proto_id']}.jcl"
    TEMPLATE_FILE = f"jcl_v2.template"
    print(f"processJclTemplate: template file = {TEMPLATE_FILE}")
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render(setup=setup,
                                 in_sample_file=in_sample_file,
                                 out_of_sample_file=out_of_sample_file
                                )
    return(outputText)
'''

In [ ]:
'''
def generatePrototypeCode(dbh,setup):

    logic = {}

    desc = formatCommentStrings(setup)
    hdr = setHeader(setup)

    logic['lsb'] = lsbLogic(setup)
    logic['poi'] = poiLogic(setup)
    logic['filters'] = filterLogic(setup)
    logic['tseg'] = timeSegmentLogic(setup)
    logic['stop_loss'] = stopLossLogic(setup)
    logic['profit_target'] = profitTargetLogic(setup)

    strat = processTemplate(hdr,desc,setup,logic)

    print(f"write Proto to {setup['strategy_file']}")
    open(setup['strategy_file'], "w").write(strat)

    return strat
'''

In [ ]:
'''
def generateJclCode(dbh,setup):
    code = processJclTemplate(setup)
    print(f"write JCL to {setup['jcl_file']}")
    open(setup['jcl_file'], "w").write(code)
    return code
'''

In [2]:
dbh = connectDB()
#proto_id = nextPrototypeId(dbh)[0].id
proto_id = nextPrototypeId(dbh)
print(f"proto_id={proto_id}")
proto_id = 2
setup = getPrototypeSetup(dbh,proto_id)
#setup['strategy_name'] = f"proto_{setup['proto_id']}_b2"


proto_id=1


In [3]:

#job_file = f'opt_job_proto_{proto_id}'
out = generatePrototypeCode(dbh,setup)
print(out)


seg=[0, 1, 2, 3]
tseg=110.0
timeframe = 15
session: 900 - 1430
1
1: add2hhmm(sess_start, tseg)=1050
1: add2hhmm(sess_start, (2*tseg) )=1240
1: add2hhmm(sess_end,-timeframe)=1415
1 done
processTemplate: template file = v4
desc={'chart_setup': 'Chart Setup:\n      symbol = @CL\n      Timeframe 0 = 15 m\n      Fitness Function = TSI\n      maxDaysBack = 200\n      session name = session 7\n      start date = 07/01/2008\n      end date = 07/01/2009', 'prototype_info': 'Prototype Info::\n      dataSet = 3\n      dataBlock = 1'}
hdr=      prototype: 2
      template: v4
      job file: /Users/szagar/ZTS/Dropbox/Business/ats/Data/OptimizationApiCode//proto_2.jcl
      poi IDs     : [1, 2, 3, 4, 5, 6, 7, 8]
      filter 1 IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
      filter 2 IDs: None
write Proto to /Users/szagar/ZTS/Dropbox/Business/ats/Data/PrototypeCode//proto_2

  
{
jinja2

In [4]:
#pprint(setup)
out = generateJclCode(dbh,setup)
print(out)


processJclTemplate: template file = jcl_v2.template
write JCL to /Users/szagar/ZTS/Dropbox/Business/ats/Data/OptimizationApiCode//proto_2.jcl

  
{ 
jinja2 generated code
Template Version: jcl_v2.template

}

using elsystem; 
using elsystem.windows.forms; 
using elsystem.drawing; 
using elsystem.collections; 
 variables: 
    Form form1(null), 
    Button StartButton(null), 
    GroupBox SymbolGroupBox(null), 
    Label PBarLabel(null), 
    TextBox SymbolToOptimizeTextBox(null),
    ProgressBar PBar(null), 
     tsopt.Optimizer optimizer(null); 
 
method void OnStartButtonClick(elsystem.Object sender, elsystem.EventArgs args) 
begin 
     
    StartOptimization(); 
    PBar.Focus(); 
end; 
 
method tsopt.Job DefineJob() 
vars: 
     
    tsopt.Job job;
begin 
	job = new tsopt.Job; 

job
  .SetOptimizationMethod(tsopt.OptimizationMethod.Genetic)
  .Securities
    .AddSecurity()
      .SetSymbol("@CL")
      .Interval
        .SetMinuteChart(15)
      .EndInterval
      .History
       